In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

In [2]:
def simulate1(n,X0=-1,Nt=2000,tf=1000,g=1.0,mu=0.0,flag_display=False):
    def model(t, X, n, g, mu):
        dXdt = np.zeros(n)
        dXdt[0] = X[0]*(1-g*X[-2]-X[0]-g*X[1]+mu*X[-3])
        dXdt[1] = X[1]*(1-g*X[-1]-X[1]-g*X[2])
        dXdt[2:n-1] = X[2:n-1]*(1-g*X[0:n-3]-X[2:n-1]-g*X[3:n])
        dXdt[-1] = X[-1]*(1-g*X[-3]-X[-1]-g*X[0])
        return dXdt
    t_span = (0, tf)
    if type(X0)==int:
        X0 = 1/(2*g+1)+0.001*np.random.rand(n)
    solution = solve_ivp(
        fun=model,
        t_span=t_span,
        y0=X0,
        args=(n, g, mu),
        method='BDF',rtol=1e-10,atol=1e-10,
        t_eval=np.linspace(t_span[0], t_span[1], Nt)
    )
    t,x = solution.t,solution.y
    if flag_display:
        plt.contour(t,np.arange(n),x,20)
        plt.xlabel('t')
        plt.ylabel('i')
    return t,x

In [3]:
def part1q1a(n, g, mu, T):
    # Define the system of ODEs
    def ODEs(x):
        dxdt = np.zeros(n)
        dxdt[0] = x[0] * (1 - x[0] - g * (x[n-2] + x[1]) + mu * x[n-3])
        dxdt[1] = x[1] * (1 - x[1] - g * (x[n-1] + x[2]))
        for i in range(2, n-2):
            dxdt[i] = x[i] * (1 - x[i] - g * (x[i-2] + x[i+1]))
        dxdt[n-1] = x[n-1] * (1 - x[n-1] - g * (x[n-3] + x[0]))
        return dxdt

    # Solve for a non-trivial equilibrium
    from scipy.optimize import root
    xbar = root(ODEs, np.ones(n) * 0.5).x  # with an initial guess

    # Construct Jacobian at the equilibrium
    J = np.zeros((n, n))
    for i in range(n):
        if i == 0:
            J[i, i] = 1 - 2 * xbar[i] - g * (xbar[n-2] + xbar[1])
            J[i, n-2] = -g * xbar[i]
            J[i, 1] = -g * xbar[i]
            J[i, n-3] = mu * xbar[i]
        elif i == 1:
            J[i, i] = 1 - 2 * xbar[i] - g * (xbar[n-1] + xbar[2])
            J[i, n-1] = -g * xbar[i]
            J[i, 2] = -g * xbar[i]
        elif i == n-1:
            J[i, i] = 1 - 2 * xbar[i] - g * (xbar[n-3] + xbar[0])
            J[i, n-3] = -g * xbar[i]
            J[i, 0] = -g * xbar[i]
        else:
            J[i, i] = 1 - 2 * xbar[i] - g * (xbar[i-2] + xbar[i+1])
            J[i, i-2] = -g * xbar[i]
            J[i, i+1] = -g * xbar[i]

    # Compute the eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(J)
    max_index = np.argmax(np.real(eigenvalues))
    xtilde0 = eigenvectors[:, max_index].real
    eratio = np.exp(2 * np.real(eigenvalues[max_index]) * T)

    return xbar, xtilde0, eratio

In [4]:
def part1q1b():
    n = 19
    g = 1.2
    mu = 2.5
    T = 50

    # Compute equilibrium solution and energy ratio using part1q1a
    xbar, _, r_a = part1q1a(n, g, mu, T)

    # Compute energy ratio from simulation
    _, x = simulate1(n=n, tf=T, g=g, mu=mu)
    e_sim = np.linalg.norm(x - xbar[:, None], axis=0)**2
    r_sim = e_sim[-1] / e_sim[0]

    return r_a, r_sim

In [ ]:
print(part1q1b())

In [6]:
def part1q1c():
    """Part 1, question 1(c): 
    Add input/output if/as needed.
    """
    #use/modify code below as needed:
    n = 19
    g = 2
    mu = 0

    # Initial setup
    T_values = np.linspace(1, 50, 50)
    eratios = []

    # Iterate over each T value with simulate1 and compute the energy ratio
    for T in T_values:
        _, x = simulate1(n, tf=T, g=g, mu=mu)
        xbar, _, _ = part1q1a(n, g, mu, T)
        e_sim = np.linalg.norm(x - xbar[:, None], axis=0)**2
        eratios.append(e_sim[-1] / e_sim[0])

    # Plot max(e(T)/e(0)) as a function of T
    plt.plot(T_values, eratios, label=r'$\max(e(t=T)/e(t=0))$')
    plt.xlabel(r'$T$')
    plt.ylabel(r'$\max(e(t=T)/e(t=0))$')
    plt.show()

In [ ]:
part1q1c()

In [8]:
def part1q2():
    # Initial setup
    g = 1
    mu = 0 
    n_values = [9, 20, 59]
    tf_values = [100, 250, 500]
    results = {}

    # Analyse each case
    for n, tf in zip(n_values, tf_values):
        # Simulate the system with final time 250
        t, x = simulate1(n=n, tf=tf, g=g, mu=mu)

        # Discard transient dynamics (first 25%)
        cutoff = int(0.25 * len(t))
        t = t[cutoff:]
        x = x[:, cutoff:]

        # Time series plot for representative components
        for i in range(5):
            plt.plot(t, x[i, :], label=f"x[{i}]")
        plt.title(f"Time Series Plot (n = {n})")
        plt.xlabel("Time")
        plt.ylabel(r'$x(t)$')
        plt.show()

        # Contour plot for all components
        plt.contour(t, np.arange(n), x, 20)
        plt.title(f"Contour Plot (n = {n})")
        plt.xlabel("Time")
        plt.ylabel("Index of State Variable")
        plt.show()

        # Observations for quantitative discussion
        results[n] = {"mean": np.mean(x, axis=1), "std": np.std(x, axis=1)}

    # Discussion and analysis
    print("Quantitative Discussion and Analysis:")
    for n, data in results.items():
        CV = np.mean(data["std"]) / np.mean(data["mean"])
        print(f"n = {n}: Coefficient of Variation = {CV}")
        if CV <= 0.1:
            print("- Dynamics appear stable and synchronised.")
        elif CV <= 0.5:
            print("- Dynamics exhibit oscillatory behaviour.")
        else:
            print("- Dynamics become complex and chaotic.")

    return None

In [ ]:
part1q2()